In [76]:
!pip install rdkit


In [77]:
import pandas as pd
import numpy as np
import rdkit as rd

In [78]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

In [109]:
df = pd.read_csv('B2AR.csv', delimiter=";")
df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type
0,CHEMBL399785,NaN,None,369.49,0,3.47,16d,CCc1nc(-c2ccc(OCCNC[C@H](O)c3cccnc3)cc2)cs1,Ki,NaN,...,Homo sapiens,SINGLE PROTEIN,CHEMBL1139766,1,Scientific Literature,Bioorg Med Chem Lett,2007.00,CHEMBL3308072,NaN,NaN
1,CHEMBL1083364,NaN,None,470.57,0,4.07,25,C=CCc1ccccc1OCC(O)CN1CCC(N2C(=O)c3cccc4cccc(c3...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1155476,1,Scientific Literature,Bioorg Med Chem Lett,2010.00,CHEMBL3307512,NaN,NaN
2,CHEMBL565547,NaN,None,293.37,0,1.71,36,Cc1nc2c3c(nc(Cc4ccccc4)n2n1)CCNCC3,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1155340,1,Scientific Literature,Bioorg Med Chem Lett,2009.00,None,NaN,NaN
3,CHEMBL1257914,NaN,None,476.60,0,4.06,10,Oc1ccc(OCC(O)CNC2CCN(c3ncnc4scc(-c5ccccc5)c34)...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1255209,1,Scientific Literature,Bioorg Med Chem Lett,2010.00,CHEMBL3308072,NaN,NaN
4,CHEMBL229429,NaN,None,260.27,0,3.02,3,N#Cc1cc(F)cc(C#Cc2csc(CF)n2)c1,Ki,'>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1145430,1,Scientific Literature,J Med Chem,2007.00,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,CHEMBL4866806,NaN,None,850.03,2,7.56,11,CCN(CCCNC[C@H](O)c1ccc(O)c2[nH]c(=O)ccc12)C(=O...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL4814011,1,Scientific Literature,Bioorg Med Chem Lett,2021.00,None,NaN,AGONIST
1704,CHEMBL4857743,NaN,None,654.81,2,5.41,8,O=C(N[C@H](c1ccccc1)c1cccc(OCCCCCCNC[C@H](O)c2...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL4814011,1,Scientific Literature,Bioorg Med Chem Lett,2021.00,None,NaN,AGONIST
1705,CHEMBL5200887,NaN,None,790.91,3,5.99,54,COc1cc(C(=O)OCCCCNC[C@H](O)c2ccc(O)c3[nH]c(=O)...,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5126603,1,Scientific Literature,J Med Chem,2022.00,None,TIME = 1.0 hr,AGONIST
1706,CHEMBL4789047,NaN,None,366.51,0,4.52,16,Cc1ccc(CN(CC(O)COc2cccc3[nH]ccc23)C(C)(C)C)cc1,Ki,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL4706660,1,Scientific Literature,J Med Chem,2021.00,None,NaN,BINDING AGENT


In [110]:
df_clean = df[['Molecule ChEMBL ID', 'Smiles', 'Target Name', 'Standard Value' ]]
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Molecule ChEMBL ID  1708 non-null   object 
 1   Smiles              1676 non-null   object 
 2   Target Name         1708 non-null   object 
 3   Standard Value      821 non-null    float64
dtypes: float64(1), object(3)
memory usage: 53.5+ KB


In [112]:
df_clean.dropna(how = 'any', inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 821 entries, 1 to 1707
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Molecule ChEMBL ID  821 non-null    object 
 1   Smiles              821 non-null    object 
 2   Target Name         821 non-null    object 
 3   Standard Value      821 non-null    float64
dtypes: float64(1), object(3)
memory usage: 32.1+ KB


In [114]:
df_clean.rename(columns = {"Molecule ChEMBL ID" : "ID", "Standard Value" : "Ki"}, inplace=True)
df_clean.head()

,ID,Smiles,Target Name,Ki
1,CHEMBL1083364,C=CCc1ccccc1OCC(O)CN1CCC(N2C(=O)c3cccc4cccc(c3...,Beta-2 adrenergic receptor,40.00
2,CHEMBL565547,Cc1nc2c3c(nc(Cc4ccccc4)n2n1)CCNCC3,Beta-2 adrenergic receptor,7600.00
3,CHEMBL1257914,Oc1ccc(OCC(O)CNC2CCN(c3ncnc4scc(-c5ccccc5)c34)...,Beta-2 adrenergic receptor,81.00
4,CHEMBL229429,N#Cc1cc(F)cc(C#Cc2csc(CF)n2)c1,Beta-2 adrenergic receptor,10000.00
12,CHEMBL1632174,O=C(O)CCCCCN1CCC(CNC(=O)c2c3n(c4ccccc24)CCCO3)CC1,Beta-2 adrenergic receptor,10000.00


In [115]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
num_descriptors = len(descriptor_names)

# Инициализация пустой матрицы для дескрипторов
descriptors_set = np.empty((0, num_descriptors), float)

# Замените DataFrame_CID на вашу переменную
for _, row in df_clean.iterrows():
    smiles = row['Smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
        descriptors = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
        descriptors_set = np.append(descriptors_set, descriptors, axis=0)

# Создание DataFrame с дескрипторами
df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)

# Объединение существующего датасета с новыми дескрипторами
df_clean = pd.concat([df_clean, df_descriptors], axis=1)

In [116]:
df_clean.head(10)

,ID,Smiles,Target Name,Ki,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
1,CHEMBL1083364,C=CCc1ccccc1OCC(O)CN1CCC(N2C(=O)c3cccc4cccc(c3...,Beta-2 adrenergic receptor,40.00,293.16,293.37,5.00,1.00,2.00,1.00,...,12.71,7.80,4.14,4.14,3.09,-2.27,13.40,5.50,2.37,3.35
2,CHEMBL565547,Cc1nc2c3c(nc(Cc4ccccc4)n2n1)CCNCC3,Beta-2 adrenergic receptor,7600.00,476.19,476.60,7.00,3.00,8.00,3.00,...,18.80,11.53,6.15,6.15,4.32,-3.20,22.59,10.38,5.14,6.89
3,CHEMBL1257914,Oc1ccc(OCC(O)CNC2CCN(c3ncnc4scc(-c5ccccc5)c34)...,Beta-2 adrenergic receptor,81.00,260.02,260.27,2.00,0.00,1.00,0.00,...,9.08,4.94,2.03,2.03,1.29,-2.24,12.22,5.39,2.91,3.66
4,CHEMBL229429,N#Cc1cc(F)cc(C#Cc2csc(CF)n2)c1,Beta-2 adrenergic receptor,10000.00,427.25,427.55,7.00,2.00,9.00,2.00,...,18.15,11.60,6.54,6.54,4.79,-2.54,21.68,10.18,5.21,7.12
12,CHEMBL1632174,O=C(O)CCCCCN1CCC(CNC(=O)c2c3n(c4ccccc24)CCCO3)CC1,Beta-2 adrenergic receptor,10000.00,419.20,419.49,8.00,2.00,5.00,2.00,...,17.43,10.49,6.08,6.08,4.38,-3.52,19.44,7.96,3.48,4.99
13,CHEMBL1242923,O=C(Cc1ccccc1)NC1CCN(CCCN2C(=O)COc3ccccc32)CC1,Beta-2 adrenergic receptor,1000.00,626.35,626.80,9.00,5.00,15.00,5.00,...,26.42,16.56,8.74,8.74,6.19,-4.18,33.19,16.94,9.42,12.22
15,CHEMBL251392,O[C@@H](CNCCOc1ccc(-c2cscn2)cc1)c1cccnc1,Beta-2 adrenergic receptor,10.00,331.18,331.41,5.00,4.00,8.00,4.00,...,14.07,8.09,4.04,4.04,2.61,-2.24,18.10,8.57,5.15,6.46
17,CHEMBL249336,O[C@@H](CNCCOc1ccc(-c2cocn2)cc1)c1cccnc1,Beta-2 adrenergic receptor,10.00,279.16,279.34,6.00,4.00,5.00,4.00,...,11.97,6.59,2.76,2.76,1.78,-1.79,14.62,5.68,3.85,4.15
18,CHEMBL3298897,O=c1ccc2c([C@@H](O)CNCCc3ccc(Nc4ccc(OCCN5CCNCC...,Beta-2 adrenergic receptor,10.00,500.06,500.34,5.00,4.00,4.00,4.00,...,15.65,9.44,5.15,5.15,3.66,-2.58,19.72,8.42,4.38,5.73
19,CHEMBL1256786,COc1ccc(CC(C)NCC(O)c2ccc(O)c(NC=O)c2)cc1,Beta-2 adrenergic receptor,33.00,464.14,464.61,7.00,3.00,13.00,3.00,...,17.29,10.28,4.88,4.88,3.23,-2.20,23.46,11.43,7.46,8.65


In [119]:
df_clean.dropna(how = 'any', inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 1 to 814
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               341 non-null    object 
 1   Smiles                           341 non-null    object 
 2   Target Name                      341 non-null    object 
 3   Ki                               341 non-null    float64
 4   exactmw                          341 non-null    float64
 5   amw                              341 non-null    float64
 6   lipinskiHBA                      341 non-null    float64
 7   lipinskiHBD                      341 non-null    float64
 8   NumRotatableBonds                341 non-null    float64
 9   NumHBD                           341 non-null    float64
 10  NumHBA                           341 non-null    float64
 11  NumHeavyAtoms                    341 non-null    float64
 12  NumAtoms              

In [120]:
df_clean

,ID,Smiles,Target Name,Ki,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
1,CHEMBL1083364,C=CCc1ccccc1OCC(O)CN1CCC(N2C(=O)c3cccc4cccc(c3...,Beta-2 adrenergic receptor,40.00,293.16,293.37,5.00,1.00,2.00,1.00,...,12.71,7.80,4.14,4.14,3.09,-2.27,13.40,5.50,2.37,3.35
2,CHEMBL565547,Cc1nc2c3c(nc(Cc4ccccc4)n2n1)CCNCC3,Beta-2 adrenergic receptor,7600.00,476.19,476.60,7.00,3.00,8.00,3.00,...,18.80,11.53,6.15,6.15,4.32,-3.20,22.59,10.38,5.14,6.89
3,CHEMBL1257914,Oc1ccc(OCC(O)CNC2CCN(c3ncnc4scc(-c5ccccc5)c34)...,Beta-2 adrenergic receptor,81.00,260.02,260.27,2.00,0.00,1.00,0.00,...,9.08,4.94,2.03,2.03,1.29,-2.24,12.22,5.39,2.91,3.66
4,CHEMBL229429,N#Cc1cc(F)cc(C#Cc2csc(CF)n2)c1,Beta-2 adrenergic receptor,10000.00,427.25,427.55,7.00,2.00,9.00,2.00,...,18.15,11.60,6.54,6.54,4.79,-2.54,21.68,10.18,5.21,7.12
12,CHEMBL1632174,O=C(O)CCCCCN1CCC(CNC(=O)c2c3n(c4ccccc24)CCCO3)CC1,Beta-2 adrenergic receptor,10000.00,419.20,419.49,8.00,2.00,5.00,2.00,...,17.43,10.49,6.08,6.08,4.38,-3.52,19.44,7.96,3.48,4.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,CHEMBL3931891,COc1cccc(C[C@H](C)NC[C@@H](O)c2cc(O)cc(O)c2)c1,Beta-2 adrenergic receptor,15881.00,363.25,363.50,6.00,4.00,7.00,4.00,...,16.18,9.55,4.36,4.36,2.91,-1.79,20.52,9.78,8.06,7.72
811,CHEMBL4113713,C[C@H](Cc1cccc(N)c1)NC[C@H](O)c1cc(O)cc(O)c1,Beta-2 adrenergic receptor,1864.00,516.21,516.73,6.00,4.00,17.00,3.00,...,20.28,11.94,5.29,5.29,3.52,-2.10,27.49,15.80,9.46,12.41
812,CHEMBL3909417,C[C@@H](Cc1cccc(N)c1)NC[C@H](O)c1cc(O)cc(O)c1,Beta-2 adrenergic receptor,6035.00,259.16,259.35,3.00,2.00,6.00,2.00,...,11.47,6.69,2.94,2.94,1.94,-1.58,13.84,6.55,3.94,4.77
813,CHEMBL4108123,CCCCC[C@@H](C)NC[C@H](O)c1cc(O)cc(O)c1,Beta-2 adrenergic receptor,9275.00,760.35,760.89,12.00,5.00,16.00,5.00,...,31.19,19.27,10.59,10.59,7.58,-5.69,38.58,18.30,9.75,12.60


In [105]:
df_clean.to_csv('b2ar_no_pr.csv', index=False)